<a href="https://www.kaggle.com/code/michalparkola/is-it-a-bird-is-it-a-plane-no-it-s-superman?scriptVersionId=138508462" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Set up the environment

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai duckduckgo_search

## Step 1: Download images of birds, planes and superman

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images_on_ddg(term, max_images=30):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
searches = 'bird','plane', 'superman'
path = Path('bird_plane_superman')

from time import sleep

for category in searches:
    print(f"Downloading images for {category}")
    dest = (path/category)
    dest.mkdir(exist_ok=True, parents=True)
    
    download_images(dest, urls = search_images_on_ddg(f'{category} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    resize_images(path/category, max_size=400, dest=path/category)

## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

## Step 3: Use the finetuned model to answer if an image is a bird, a plane or superman.

In [ ]:
category,_,probs = learn.predict(PILImage.create('superman.jpg'))
print(f"This is a: {category}.")
print(f"Probabilities: {probs}")

In [ ]:
images = get_image_files(path)
image = PILImage.create(images[17])

category,_,probs = learn.predict(image)
print(f"This is a: {category}.")
print(f"Probabilities: {max(probs):.4f}")